In [5]:
import numpy as np
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt

from neuprint import Client
from neuprint import fetch_neurons
from neuprint import fetch_adjacencies
from neuprint import NeuronCriteria as NC

In [7]:
# Initialize client
token = '' # From https://neuprint.janelia.org/account
c = Client('neuprint.janelia.org', dataset='manc:v1.0', token=token)
c.fetch_version()

'0.1.0'

In [8]:
# IDs of chief 9A neurons
chief_9A_ids = [100513, # L1
                13157, # L2
                14517, # L3
                165560, # R1
                12443, # R2
                12804] # R3

In [9]:
# IDs of hook neurons
criteria = NC(type='SNpp38')
neuron_df, roi_counts_df = fetch_neurons(criteria)
hook_ids = neuron_df.bodyId.to_list()

C:\Users\Chris\anaconda3\envs\Dallmann_2024\lib\site-packages\neuprint\client.py:609: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]
C:\Users\Chris\anaconda3\envs\Dallmann_2024\lib\site-packages\neuprint\client.py:619: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  neuron_props_val = df_results.iloc[0][0]


In [111]:
# Selecct specific chief 9A
chief_9A_id = chief_9A_ids[0]

In [112]:
# Find postsynaptic neurons
neuron_df, conn_df = fetch_adjacencies(chief_9A_id, None)

In [113]:
# Sum weights across neuropils and sort
conn_df = conn_df.groupby(['bodyId_pre','bodyId_post'])['weight'].sum().reset_index().sort_values('weight', ascending=False)

In [114]:
# Remove connections with weights < weight_thresh
weight_thresh = 5 
idx_drop = conn_df.index[conn_df.weight<weight_thresh]
conn_df = conn_df.drop(index=idx_drop) 

In [115]:
# Find type of partner neurons  
neuron_type = [] 
n_syn_hook = 0
for idx, row in conn_df.iterrows():
    neuron_type_temp = neuron_df.type[neuron_df.bodyId == conn_df.bodyId_post[idx]].item()
    if neuron_type_temp is None:
        neuron_type.append('Unknown')
    elif 'IN' in neuron_type_temp:
        neuron_type.append('IN')
    elif 'MN' in neuron_type_temp:
        neuron_type.append('MN')
    elif 'AN' in neuron_type_temp:
        neuron_type.append('AN')
    elif 'DN' in neuron_type_temp:
        neuron_type.append('DN')
    elif 'SN' in neuron_type_temp or 'SA' in neuron_type_temp:
        neuron_type.append('SN')
        if conn_df.bodyId_post[idx] in hook_ids:
            n_syn_hook += conn_df.weight[idx].item()
    elif 'EN' in neuron_type_temp or 'EA' in neuron_type_temp:
        # The web neuron makes only few synapses with efferent neurons (ENs). They are counted as IN here.
        neuron_type.append('IN') 
        
conn_df['neuron_type'] = neuron_type

In [116]:
# Sum weights for each type 
conn_df_sum = conn_df.groupby(['neuron_type'])['weight'].sum().reset_index().sort_values('weight', ascending=False)
conn_df_sum

,neuron_type,weight
3,SN,602
1,IN,227
4,Unknown,111
2,MN,13
0,AN,6


In [117]:
# Calculate relative weight for specific type 
conn_df_sum.weight[conn_df_sum.neuron_type=='SN'].item() / conn_df_sum.weight.sum()

0.6277372262773723

In [118]:
# Calculate relative weight for hook 
n_syn_hook / conn_df_sum.weight.sum()

0.4942648592283629